# M-GEP

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import random
import copy

In [2]:
dataset = 'kepsilon'

# Loading data - select which cases to include in the training/validation set (commented out cases are held out)
cases = ['DUCT_1100',
         'DUCT_1150',
         'DUCT_1250',
         'DUCT_1300',
         'DUCT_1350',
         'DUCT_1400',
         'DUCT_1500',
         'DUCT_1600',
         'DUCT_1800',
         #'DUCT_2000',
         'DUCT_2205',
         'DUCT_2400',
         'DUCT_2600',
         'DUCT_2900',
         'DUCT_3200',
         #'DUCT_3500',
         'PHLL_case_0p5',
         'PHLL_case_0p8',
         'PHLL_case_1p0',
         #'PHLL_case_1p2',
         'PHLL_case_1p5',
         'BUMP_h20',
         'BUMP_h26',
         'BUMP_h31',
         #'BUMP_h38',
         'BUMP_h42',
         'CNDV_12600',
         'CNDV_20580',
         'CBFS_13700'
         ]

In [3]:
#Convenient functions for loading dataset
def loadCombinedArray(cases,field):
    data = np.concatenate([np.load('../data/'+dataset+'/'+dataset+'_'+case+'_'+field + '.npy', allow_pickle=True) for case in cases])
    return data

In [4]:
def loadLabels(cases,field):
    data = np.concatenate([np.load('../data/'+'labels/'+case+'_'+field + '.npy') for case in cases])
    return data

In [5]:
print('Loading features and labels from the dataset: '+ dataset)

Loading features and labels from the dataset: kepsilon


In [6]:
#Load the set of ten basis tensors (N,10,3,3), from Pope "A more general effective-viscosity hypothesis" (1975).
Tensors = loadCombinedArray(cases,'Tensors')
print('Shape of basis tensor array: '+str(Tensors.shape))

Shape of basis tensor array: (791490, 10, 3, 3)


In [7]:
#Load the 47 invariants (N,47) used by Wu et al. "Physics-informed machine learning approach for augmenting turbulence models: A comprehensive framework" (2018)
Invariants = loadCombinedArray(cases,'I1')
print('Shape of invariant features array: '+str(Invariants.shape))

Shape of invariant features array: (791490, 47)


In [8]:
# #Load the additional scalars (N,5): 
# Scalars = loadCombinedArray(cases,'q')
# print('Shape of scalar features array: '+str(Scalars.shape))

In [9]:
Features = Invariants.copy()
print('Shape of combined features array: '+str(Features.shape))

Shape of combined features array: (791490, 47)


# Data Processing

In [10]:
def remove_outliers(Features):
    stdev = np.std(Features,axis=0)
    means = np.mean(Features,axis=0)
    ind_drop = np.empty(0)
    for i in range(len(Features[0,:])):
        ind_drop = np.concatenate((ind_drop,np.where((Features[:,i]>means[i]+5*stdev[i]) | (Features[:,i]<means[i]-5*stdev[i]) )[0]))
    return ind_drop.astype(int)

outlier_removal_switch = 0
if outlier_removal_switch == 1:
    outlier_index = remove_outliers(Features)
    print('Found '+str(len(outlier_index))+' outliers in the input feature set')
    Features = np.delete(Features,outlier_index,axis=0)
    Tensors = np.delete(Tensors,outlier_index,axis=0)
    Labels = np.delete(Labels,outlier_index,axis=0)


In [11]:
#Load the label set from DNS/LES:
Labels = loadLabels(cases,'b')
#If desired, reshape the 3x3 symmetric anisotropy tensor into a 1x6 vector
# Labels = np.delete(Labels.reshape((len(Labels),9)),[3,6,7],axis=1)
print('Shape of DNS/LES labels array: '+str(Labels.shape))

Shape of DNS/LES labels array: (791490, 3, 3)


In [12]:
indices = np.arange(Features.shape[0])

In [13]:
x_train, x_val, y_train, y_val, ind_train, ind_val = train_test_split(Features, Labels, indices, test_size=0.2, random_state=10, shuffle=True)

In [14]:
basis_train = Tensors[ind_train]
basis_val = Tensors[ind_val]

In [15]:
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_val = scaler.transform(x_val)

In [16]:
print(' ')
print('Training features:')
print(x_train.shape)
print('Training tensor basis:')
print(basis_train.shape)
print('Training labels:')
print(y_train.shape)
print(' ')
print('Validation features:')
print(x_val.shape)
print('Validation tensor basis:')
print(basis_val.shape)
print('Validation labels:')
print(y_val.shape)
print(' ')

 
Training features:
(633192, 47)
Training tensor basis:
(633192, 10, 3, 3)
Training labels:
(633192, 3, 3)
 
Validation features:
(158298, 47)
Validation tensor basis:
(158298, 10, 3, 3)
Validation labels:
(158298, 3, 3)
 


# GEP

In [18]:
def dot_prod(A, B):
    return np.array([np.dot(a, b) for a, b in zip(A, B)])

def transpose(a):
    return np.transpose(a, axes=[0,2,1])

In [19]:
class Operator:
    def __init__(self, name, func):
        self.name = name
        self.func = func
        self.nargs = 2
        if self.name in ['T', 's', 'e']:
            self.nargs = 1
    def __call__(self, l, r=None):
        if self.nargs == 1:
            return self.func(l)
        return self.func(l, r)

class Terminal:
    def __init__(self, name, value):
        self.name = name
        self.value = value

# generate function set for chromosome
def generate_F():
    plus = Operator('+', np.add)
    minus = Operator('-', np.subtract)
    dot = Operator('.', dot_prod)
    tp = Operator('T', transpose)
    return [plus, minus, dot, tp]

# generate terminal set for chromosome
def generate_T():
    Aij = Terminal('Aij', np.random.rand(2,3,3))
    Bij = Terminal('Bij', np.random.rand(2,3,3))
    I = Terminal('I', np.array([np.identity(3), np.identity(3)]))
    RNCij = Terminal('RNCij', np.random.rand(2, 3, 3))
    return [Aij, Bij, I, RNCij]

# generate function set for plasmid
def generate_Fp():
    plus = Operator('+', np.add)
    minus = Operator('-', np.subtract)
    star = Operator('*', np.multiply)
    sin = Operator('s', np.sin)
    exp = Operator('e', np.exp)
    return [plus, minus, star, sin, exp]

# generate terminal set for plasmid
def generate_Tp():
    a = Terminal('a', [10, 12])
    b = Terminal('b', [20, 22])
    one = Terminal('1', [1, 1])
    q = Terminal('?', [5,6])
    return [a, b, one, q]

In [20]:
def generate_head(h, set_FT, pls_params=None):
    head = []
    for i in range(h):
        gene = np.random.choice(set_FT)
        if gene == 'p':
            gene = Plasmid(pls_params)
        head.append(gene)
    return np.array(head)

def generate_tail(t, T):
    tail = []
    for i in range(t):
        gene = np.random.choice(T)
        tail.append(gene)
    return np.array(tail)

class Chromosome:
    def __init__(self, chr_params, pls_params):
        self.h = chr_params['h']
        self.t = chr_params['t']
        self.F = chr_params['F']
        self.T = chr_params['T']
        self.P = chr_params['P']
        self.head = generate_head(self.h, self.F + self.T + self.P, pls_params)
        self.tail = generate_tail(self.t, self.T)
        self.pls_params = pls_params
    
    def calc(self, head, tail):
        if len(head) != 0:
            gene = head[0]
            if type(gene) == Terminal:
                return gene.value, head[1:], tail
            elif type(gene) == Operator:
                if gene.nargs == 2:
                    left, new_head, new_tail = self.calc(head[1:], tail)
                    right, new_head, new_tail = self.calc(new_head, new_tail)
                    return gene(left, right), new_head, new_tail
                elif gene.nargs == 1:
                    left, new_head, new_tail = self.calc(head[1:], tail)
                    return gene(left), new_head, new_tail
            elif type(gene) == Plasmid:
                left, new_head, new_tail = self.calc(head[1:], tail)
                return gene(left), new_head, new_tail
        else:
            gene = tail[0]
            return gene.value, head, tail[1:]

    def forward(self):
        value, _, _ = self.calc(self.head, self.tail)
        return value

class Plasmid:
    def __init__(self, pls_params):
        self.hp = pls_params['hp']
        self.tp = pls_params['tp']
        self.Fp = pls_params['Fp']
        self.Tp = pls_params['Tp']
        self.head = generate_head(self.hp, self.Fp + self.Tp)
        self.tail = generate_tail(self.tp, self.Tp)
        self.value = self.evaluate()
        self.name = 'plasmid'

    def __call__(self, x):
        return np.multiply(self.value.reshape((-1, 1, 1)), x)
    
    def __calc(self, head, tail):
        if len(head) == 0:
            gene = tail[0]
            # here gene type is always Terminal, however, we keep it for future
            if type(gene) == Terminal:
                return gene.value, head, tail[1:]
            elif type(gene) == Operator:
                if gene.nargs == 2:
                    left, new_head, new_tail = self.__calc(head, tail[1:])
                    right, new_head, new_tail = self.__calc(new_head, new_tail)
                    return gene(left, right), new_head, new_tail
                elif gene.nargs == 1:
                    left, new_head, new_tail = self.__calc(head, tail[1:])
                    return gene(left), new_head, new_tail
        else:
            gene = head[0]
            if type(gene) == Terminal:
                return gene.value, head[1:], tail
            elif type(gene) == Operator:
                if gene.nargs == 2:
                    left, new_head, new_tail = self.__calc(head[1:], tail)
                    right, new_head, new_tail = self.__calc(new_head, new_tail)
                    return gene(left, right), new_head, new_tail
                elif gene.nargs == 1:
                    left, new_head, new_tail = self.__calc(head[1:], tail)
                    return gene(left), new_head, new_tail
    def evaluate(self):
        value, _, _ = self.__calc(self.head, self.tail)
        return np.array(value).flatten()

In [35]:
class Population:
    def __init__(self, N, chr_params, pls_params):
        self.N = N
        self.chr_params = chr_params
        self.pls_params = pls_params
        self.chromosomes = self.generate_population(self.N)

    def generate_population(self, N):
        chromosomes = []
        for i in range(N):
            individ = Chromosome(self.chr_params, self.pls_params)
            chromosomes.append(individ)
        return np.array(chromosomes)
    
    def compute_fitness(self, y_true):
        values = []
        for chromosome in self.chromosomes:
            value = chromosome.forward()
            values.append(value)
        fitness = np.abs(np.sum(values - y_true, axis=(1,2,3)))
        return fitness
    
    def select_fittest(self, fitness, frac=0.5):
        median = np.median(fitness)
        selection = fitness < median
        return self.chromosomes[selection]
    
    def mutate_chromosome(self, chromosome):
        n_genes = len(chromosome.head)
        idx = np.random.randint(0, n_genes)
        chromosome.head[idx] = np.random.choice(chromosome.F + chromosome.T + chromosome.P)
        if chromosome.head[idx] == 'p':
            chromosome.head[idx] = Plasmid(chromosome.pls_params)
        return chromosome
    
    def mutate_population_chromosomes(self, chromosomes, p=0.05):
        size = len(chromosomes)
        mutation_indices = np.random.choice(np.arange(size), int(size*p))
        for i in mutation_indices:
            chromosomes[i] = self.mutate_chromosome(chromosomes[i])
        return chromosomes
    
    def crossover(self, left, right):
        h = len(left.head)
        t = len(left.tail)

        mask = np.random.choice([0, 1], size=(h,))
        left.head[mask], right.head[mask] = right.head[mask], left.head[mask]

        mask = np.random.choice([True, False], size=(t,))
        left.tail[mask], right.tail[mask] = right.tail[mask], left.tail[mask]

        return left, right
    
    def crossover_chromosomes(self, chromosomes):
        mid = int(len(chromosomes)/2)
        for i in range(mid):
            chromosomes[i], chromosomes[-i-1] = self.crossover(chromosomes[i], chromosomes[-i-1])
        return chromosomes
    
    def mutate_plasmid(self, plasmid):
        head_len = len(plasmid.head)
        tail_len = len(plasmid.tail)

        if np.random.rand() < head_len / (head_len + tail_len):
            idx = np.random.randint(0, head_len)
            plasmid.head[idx] = np.random.choice(plasmid.Fp + plasmid.Tp)
            return plasmid
        else:
            idx = np.random.randint(0, tail_len)
            plasmid.tail[idx] = np.random.choice(plasmid.Tp)
            return plasmid
    
    def mutate_population_plasmids(self, plasmids, p=0.05):
        size = len(plasmids)
        mutation_indices = np.random.choice(np.arange(size), int(size*p))
        for i in mutation_indices:
            plasmids[i] = mutate_plasmid(plasmids[i])
        return plasmids
    
    def crossover_plasmids(self, plasmids):
        mid = int(len(plasmids)/2)
        for i in range(mid):
            plasmids[i], plasmids[-i-1] = self.crossover(plasmids[i], plasmids[-i-1])
        return plasmids
    
    def collect_plasmids(self, chromosomes):
        plasmids = []
        for i, chromosome in enumerate(chromosomes):
            for j, gene in enumerate(chromosome.head):
                if type(gene) == Plasmid:
                    plasmids.append(gene)
        return plasmids
    
    def redistribute_plasmids(self, plasmids, chromosomes):
        i = 0
        for chromosome in chromosomes:
            for j, gene in enumerate(chromosome.head):
                if type(gene) == Plasmid:
                    chromosome.head[j] = plasmids[i]
                    i += 1
        return chromosomes

    def iteration(self, y_true):
        fitness = self.compute_fitness(y_true)
        elite_chromosome = copy.deepcopy(self.chromosomes[np.argmin(fitness)])
        self.chromosomes = np.delete(self.chromosomes, np.argmin(fitness))
        fitness = np.delete(fitness, np.argmin(fitness))
        new_generation = self.select_fittest(fitness)
        new_generation = self.mutate_population_chromosomes(new_generation)
        new_generation = self.crossover_chromosomes(new_generation)
        
        plasmids = self.collect_plasmids(new_generation)
        plasmids = self.mutate_population_plasmids(plasmids)
        plasmids = self.crossover_plasmids(plasmids)
        new_generation = self.redistribute_plasmids(plasmids, new_generation)

        extension = self.generate_population(self.N - len(new_generation) - 1)
        
        return np.concatenate(([elite_chromosome], new_generation, extension))
    
    def train(self, epochs, y_true):
        for i in range(epochs):
            self.chromosomes = self.iteration(y_true)
            print(i)

In [36]:
h = 3
t = h * (2 - 1) + 1
hp = 3
tp = hp * (2 - 1) + 1
N = 100
F = generate_F()
T = generate_T()
Fp = generate_Fp()
Tp = generate_Tp()
P = ['p']

chr_params = {'h': h, 't': t, 'F': F, 'T': T, 'P': P}
pls_params = {'hp': hp, 'tp': tp, 'Fp': Fp, 'Tp': Tp}

y_true = np.ones((2, 3, 3))


pop = Population(N, chr_params, pls_params)
pop.train(10, y_true)

0
1
2
3
4
5
6
7
8
9


/tmp/ipykernel_4784/705305288.py:10: RuntimeWarning: overflow encountered in exp
  return self.func(l)


In [39]:
pop.chromosomes[0]